<a href="https://colab.research.google.com/github/whkwls2653/Kaggle_JaneStreet/blob/main/JaneStreet_simpleMLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jwajinlee","key":"e2e37a864e7b7b20607dfa768d10627e"}'}

In [8]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록 
!chmod 600 ~/.kaggle/kaggle.json
# 본인이 참가한 모든 대회 보기 
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes        107           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes        216           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        425           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       2900           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      22403           False  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started  Knowledge       6004           False  
connectx

In [32]:
%cd Kaggle/janestreet/
!ls
import janestreet

[Errno 2] No such file or directory: 'Kaggle/janestreet/'
/content/gdrive/MyDrive/Kaggle/janestreet
competition.cpython-37m-x86_64-linux-gnu.so  features.csv  sample_data
encoder.hdf5				     __init__.py   simple_mlp
example_sample_submission.csv		     kaggle.json   train.csv
example_test.csv.zip			     __pycache__   train.csv.zip


ModuleNotFoundError: ignored

In [29]:
for dirname, _, filenames in os.walk('/content/gdrive/MyDrive/Kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import janestreet

/content/gdrive/MyDrive/Kaggle/__init__.py
/content/gdrive/MyDrive/Kaggle/competition.cpython-37m-x86_64-linux-gnu.so
/content/gdrive/MyDrive/Kaggle/train.csv.zip
/content/gdrive/MyDrive/Kaggle/example_sample_submission.csv
/content/gdrive/MyDrive/Kaggle/example_test.csv.zip
/content/gdrive/MyDrive/Kaggle/features.csv
/content/gdrive/MyDrive/Kaggle/janestreet/example_test.csv.zip
/content/gdrive/MyDrive/Kaggle/janestreet/train.csv.zip
/content/gdrive/MyDrive/Kaggle/janestreet/example_sample_submission.csv
/content/gdrive/MyDrive/Kaggle/janestreet/features.csv
/content/gdrive/MyDrive/Kaggle/janestreet/train.csv
/content/gdrive/MyDrive/Kaggle/janestreet/kaggle.json
/content/gdrive/MyDrive/Kaggle/janestreet/__init__.py
/content/gdrive/MyDrive/Kaggle/janestreet/competition.cpython-37m-x86_64-linux-gnu.so
/content/gdrive/MyDrive/Kaggle/janestreet/encoder.hdf5
/content/gdrive/MyDrive/Kaggle/janestreet/sample_data/README.md
/content/gdrive/MyDrive/Kaggle/janestreet/sample_data/anscombe.json
/

ModuleNotFoundError: ignored

In [15]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices
import os

SEED = 1111

tf.random.set_seed(SEED)
np.random.seed(SEED)


save=False
resume=True
train = pd.read_csv('/content/gdrive/MyDrive/Kaggle/janestreet/train.csv')
train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]

###NaN 값을 mean으로 채워 넣고 있음. EDA 본 결과 이러면 문제가 많을 것 같음.

train.fillna(train.mean(),inplace=True)

# action값을 resp가 0이상 -> 가격이 올랐을때 1로 새로운 열을 만들어 채워 넣음

train['action'] = ((train['resp'].values) > 0).astype(int)


features = [c for c in train.columns if "feature" in c]

f_mean = np.mean(train[features[1:]].values,axis=0)

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

X_train = train.loc[:, train.columns.str.contains('feature')]
#y_train = (train.loc[:, 'action'])

y_train = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T




def create_mlp(
    num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
):

    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)

    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics=tf.keras.metrics.AUC(name="AUC"),
    )

    return model


batch_size = 5000
hidden_units = [160, 160, 160]
dropout_rates = [0.25, 0.25, 0.25, 0.25]
label_smoothing = 1e-2
learning_rate = 1e-3

if save==True:
  checkpoint_path = "/content/gdrive/MyDrive/Kaggle/janestreet/simple_mlp/cp.ckpt"
  checkpoint_dir = os.path.dirname(checkpoint_path)
  cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)





clf = create_mlp(
    len(features), 5, hidden_units, dropout_rates, label_smoothing, learning_rate
    )

if resume==True:
  clf.load_weights(checkpoint_path)
else:
  clf.fit(X_train, y_train, epochs=200, batch_size=5000,callbacks=[cp_callback] )


models = []

models.append(clf)

th = 0.5000


f = np.median
models = models[-3:]
import janestreet
env = janestreet.make_env()
for (test_df, pred_df) in tqdm(env.iter_test()):
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, features].values
        if np.isnan(x_tt[:, 1:].sum()):
            x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
        pred = np.mean([model(x_tt, training = False).numpy() for model in models],axis=0)
        pred = f(pred)
        pred_df.action = np.where(pred >= th, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)

Epoch 1/200
315/315 [==============================] - 5s 12ms/step - loss: 0.7118 - AUC: 0.5151

Epoch 00001: saving model to /content/gdrive/MyDrive/Kaggle/janestreet/simple_mlp/cp.ckpt
Epoch 2/200
315/315 [==============================] - 4s 12ms/step - loss: 0.6913 - AUC: 0.5330

Epoch 00002: saving model to /content/gdrive/MyDrive/Kaggle/janestreet/simple_mlp/cp.ckpt
Epoch 3/200
315/315 [==============================] - 4s 12ms/step - loss: 0.6904 - AUC: 0.5378

Epoch 00003: saving model to /content/gdrive/MyDrive/Kaggle/janestreet/simple_mlp/cp.ckpt
Epoch 4/200
315/315 [==============================] - 4s 12ms/step - loss: 0.6900 - AUC: 0.5409

Epoch 00004: saving model to /content/gdrive/MyDrive/Kaggle/janestreet/simple_mlp/cp.ckpt
Epoch 5/200
315/315 [==============================] - 4s 12ms/step - loss: 0.6898 - AUC: 0.5421

Epoch 00005: saving model to /content/gdrive/MyDrive/Kaggle/janestreet/simple_mlp/cp.ckpt
Epoch 6/200
315/315 [==============================] - 4s 12

ModuleNotFoundError: ignored